In [1]:
import os
import pandas as pd
import boto3
from botocore.exceptions import ClientError
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import shutil

In [2]:
def get_file(bucket, remote_path, local_path):
    try:
        bucket.download_file(remote_path, local_path)
        return True
    except:
        return False

In [3]:
QUANTIGO_BUCKET = 'quantigo'

In [4]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(QUANTIGO_BUCKET)

remote_zip_files = [obj.key
                    for obj in bucket.objects.filter(Prefix='frame')
                    if obj.key.endswith('.zip')]

remote_zip_files[:3], remote_zip_files[-3:]

(['frames_clubber_video1-000027-000070.zip',
  'frames_clubber_video1-000133-000167.zip',
  'frames_clubber_video1-000183-000184.zip'],
 ['frames_clubber_video9-004135-004143.zip',
  'frames_clubber_video9-004158-004177.zip',
  'frames_clubber_video9-004178-004219.zip'])

In [5]:
n_images = {'match': [], 'n': []}

for zip_file in remote_zip_files:
    
    if not get_file(bucket, zip_file, zip_file):
        raise Exception('Unable to download ' + zip_file)
    
    if os.system('unzip ' + zip_file) != 0:
        raise Exception('Unable to unzip ' + zip_file)
    
    n_images['match'].append(zip_file.split('.')[0])
    n_images['n'].append(len(os.listdir(zip_file.split('.')[0])))
    
    os.remove(zip_file)
    shutil.rmtree(zip_file.split('.')[0])                

In [7]:
df = pd.DataFrame(n_images)
df

,match,n
0,frames_clubber_video1-000027-000070,72
1,frames_clubber_video1-000133-000167,57
2,frames_clubber_video1-000183-000184,2
3,frames_clubber_video1-000191-000197,10
4,frames_clubber_video1-000237-000268,52
...,...,...
1718,frames_clubber_video9-004054-004079,42
1719,frames_clubber_video9-004102-004123,35
1720,frames_clubber_video9-004135-004143,14
1721,frames_clubber_video9-004158-004177,32


In [8]:
df['n'].sum()

47399